In [37]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import MinMaxScaler

In [38]:
df = pd.read_csv("C:/Training/Academy/Statistics (Python)/Cases/human-resources-analytics/HR_comma_sep.csv")
dum_df = pd.get_dummies(df)
dum_df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department_IT,Department_RandD,...,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,False,False,...,False,False,False,False,True,False,False,False,True,False
1,0.80,0.86,5,262,6,0,1,0,False,False,...,False,False,False,False,True,False,False,False,False,True
2,0.10,0.77,6,247,4,0,1,0,False,False,...,False,False,False,False,True,False,False,False,True,False
3,0.92,0.85,5,259,5,0,1,0,False,False,...,False,False,False,False,True,False,False,False,True,False
4,0.89,1.00,5,224,5,0,1,0,False,False,...,False,False,False,False,True,False,False,False,True,False


In [39]:
X = dum_df.drop('left', axis=1)
scaler = MinMaxScaler()

y = dum_df['left'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=23,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [40]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([10496, 20])
torch.Size([10496])


In [41]:
type(X_torch)

torch.Tensor

In [42]:
X_scl_trn.shape[1]

20

In [43]:
torch.manual_seed(23)
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=15),
                      nn.ReLU(),
                      nn.Linear(15, 8 ),
                      nn.ReLU(),
                      nn.Linear(8, 5),
                      nn.ReLU(),
                      nn.Linear(5,1),
                      nn.Sigmoid())

In [44]:
model.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=20, out_features=15, bias=True)
  (1): ReLU()
  (2): Linear(in_features=15, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=1, bias=True)
  (7): Sigmoid()
)>

In [45]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [46]:
y_pred = model(X_torch.float())
print(y_torch.shape)

torch.Size([10496])


In [47]:
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([10496, 1])
torch.Size([10496, 1])


In [48]:
y_torch.size()

torch.Size([10496, 1])

Prediction with initialized weights

In [49]:
y_pred[:5]

tensor([[0.4525],
        [0.4561],
        [0.4490],
        [0.4524],
        [0.4523]], grad_fn=<SliceBackward0>)

### Initial Log Loss

In [50]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(0.6471, grad_fn=<BinaryCrossEntropyBackward0>)

In [51]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.6471357345581055
epoch:  101  loss:  0.5489823222160339
epoch:  201  loss:  0.5477492213249207
epoch:  301  loss:  0.5470072031021118
epoch:  401  loss:  0.5458980202674866
epoch:  501  loss:  0.5438758730888367
epoch:  601  loss:  0.5396278500556946
epoch:  701  loss:  0.5296365022659302
epoch:  801  loss:  0.503637433052063
epoch:  901  loss:  0.45192766189575195


### Training Set Log Loss

In [52]:
loss

tensor(0.4212, grad_fn=<BinaryCrossEntropyBackward0>)

In [53]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [54]:
y_pred_prob = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(y_pred_prob)

torch.Tensor

In [55]:
y_pred_prob

tensor([[0.2034],
        [0.3240],
        [0.2160],
        ...,
        [0.0473],
        [0.5336],
        [0.0837]], grad_fn=<SigmoidBackward0>)

In [56]:
y_pred_prob = y_pred_prob.detach().numpy()
type(y_pred_prob)

numpy.ndarray

In [57]:
y_pred_prob.shape

(4499, 1)

In [58]:
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob.shape

(4499,)

In [59]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)

y_pred[:5]

array([0, 0, 0, 0, 0])

### Test Set Accuracy Score

In [60]:
print(accuracy_score(y_test,y_pred))

0.8139586574794399


### Test Set Log Loss

In [61]:
log_loss(y_test, y_pred_prob)

0.42473838274422615